In [95]:
#!/usr/bin/env python3
"""
Patient Timeline Creator
Creates a comprehensive timeline for patients by combining data from multiple tables.
"""

import pandas as pd
import duckdb as db
import os
from datetime import datetime
from typing import Dict, List, Tuple, Optional

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [101]:

def get_database_connection():
    """Create and return a connection to the clinisys_all database"""
    path_to_db = '../../database/huntington_data_lake.duckdb'
    conn = db.connect(path_to_db, read_only=True)
    
    print(f"Connected to database: {path_to_db}")
    print(f"Database file exists: {os.path.exists(path_to_db)}")
    
    return conn

def extract_timeline_data_for_patient(conn, test_prontuario):
    """Extract timeline data for a specific patient"""
    query = f"""
        SELECT * FROM gold.all_patients_timeline
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def get_finops_data(conn, test_prontuario):
    """Get finops data for a specific patient"""
    query = f"""
        SELECT * FROM gold.finops_summary
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def get_mescladas(conn, test_prontuario):
    """Get mescladas data for a specific patient"""
    query = f"""
        SELECT * FROM silver.mesclada_vendas
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def main(test_prontuario):
    """Main function to create patient timeline"""
    
    # Test with a single patient first
    # test_prontuario = 175583
    
    print(f"Creating timeline for patient: {test_prontuario}")
    
    # Get database connection
    conn = get_database_connection()
    
    try:
        # Extract timeline data
        timeline_df = extract_timeline_data_for_patient(conn, test_prontuario)
       
        finops_df = get_finops_data(conn, test_prontuario)

        mescladas_df = get_mescladas(conn, test_prontuario)
        return timeline_df, finops_df, mescladas_df
        
    finally:
        conn.close()

In [102]:
if __name__ == "__main__":
    patient_id = 876_950
    patient_id = 825_890 # Claudia
    patient_id = 175_583 # Renata
    # patient_id = 220_783 # caso cabelud0:: - 26 tentativas
    # patient_id = 182_925 # caso completo
    patient_id = 162_173
    patient_id = 641_950 # 11 cycles without payment
    patient_id = 150_491 # ovo recepção
    # patient_id = 155_282 # medico externo
    # patient_id = 738_349 # (marido 738_350 não aparece no clinisys) -- sem pagamento
    # patient_id = 678_207 # numero cliente duplicado no diario_vendas (78207)
    # patient_id = 839574 # utero de substituicao (815299)
    # patient_id = 849697 # 
    patient_id = 842663 # doadora
    # patient_id = 832556
    # patient_id = 873794
    patient_id = 172179

    timeline_df, finops_df, mescladas_df = main(patient_id)


Creating timeline for patient: 172179
Connected to database: ../../database/huntington_data_lake.duckdb
Database file exists: True


In [103]:
timeline_df

,prontuario,event_id,event_date,reference,reference_value,tentativa,unidade,resultado_tratamento,flag_date_estimated,additional_info
0,172179.0,1465505,2025-08-10,extrato_atendimentos,Congelamento Seminal: Ciclo,None,1. HTT SP - Ibirapuera,None,False,{}
1,172179.0,1444229,2025-07-30,extrato_atendimentos,Grupo apoio Psicologia,None,2. HTT SP - Vila Mariana,None,False,{}
2,172179.0,1438753,2025-07-06,extrato_atendimentos,Espermograma para Ciclo,None,1. HTT SP - Ibirapuera,None,False,{}
3,172179.0,35317,2025-06-26,tratamentos,Ciclo a Fresco FIV,1,1. HTT SP - Ibirapuera,No transfer,False,{}
4,172179.0,14280,2025-06-26,congelamentos_ovulos,2183/25,None,None,None,False,"{""NOvulos"": ""8"", ""Unidade"": ""1""}"
5,172179.0,1431618,2025-06-26,extrato_atendimentos,Coleta Seminal: FIV,None,1. HTT SP - Ibirapuera,None,False,{}
6,172179.0,1431298,2025-06-26,extrato_atendimentos,Aspiração Folicular: FIV,None,1. HTT SP - Ibirapuera,None,False,{}
7,172179.0,1432126,2025-06-24,extrato_atendimentos,6º US Ciclo,None,1. HTT SP - Ibirapuera,None,False,{}
8,172179.0,1431342,2025-06-23,extrato_atendimentos,6º US Ciclo,None,1. HTT SP - Ibirapuera,None,False,{}
9,172179.0,1430041,2025-06-21,extrato_atendimentos,5º US Ciclo,None,1. HTT SP - Ibirapuera,None,False,{}


In [104]:
finops_df

,prontuario,cycle_with_transfer,cycle_without_transfer,cycle_total,treatment_paid_count,treatment_paid_total,timeline_first_date,timeline_last_date,billing_first_date,billing_last_date,flag_is_donor,flag_cycles_no_payments,flag_more_cycles_than_payments,flag_no_cycles_but_payments,flag_less_cycles_than_payments,timeline_unidade
0,172179.0,0,1,1,0.0,0.0,2025-06-26,2025-06-26,NaT,NaT,0,1,1,0,0,1. HTT SP - Ibirapuera


In [100]:
timeline_df.loc[5, 'additional_info']

'{}'

In [105]:
mescladas_df

,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente.1,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
0,74126,JULIANA ALVES,74126,JULIANA ALVES DE LIMA,172179,2025-09-02,MENSALIDADE CRIOPRESERVACAO DE OOCITOS -,1.0,65.5,Criopreservação,10150,502,30045,RPS,29833,0,CRIOPRESERVACAO,801466,,S3566.40,0.0,0.0,0.0,59.5,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,1529,20250916_153309,Mescladas2.xlsx
1,74126,JULIANA ALVES,74126,JULIANA ALVES DE LIMA,172179,2024-05-31,CONSULTA ACESSO INSTITUTO,1.0,40.0,Outras Consultas,10104,502,98733,RPS,97949,73,MICHELE PANZAN,801466,,S3557.04,0.0,0.0,0.0,0.0,Vila Mariana,0,0,31110100011,,,Consultas,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,247027,20250916_153309,Mescladas2.xlsx
2,74126,JULIANA ALVES,74126,JULIANA ALVES DE LIMA,172179,2025-06-06,PRIMOGYNA 2MG C 28 CP (C.E.),1.0,70.0,Estimulação,10150,502,14638,RPS,14566,73,MICHELE PANZAN,801466,,000000000004645,0.0,0.0,0.0,48.0,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,380362,20250916_153309,Mescladas2.xlsx
3,74126,JULIANA ALVES,74126,JULIANA ALVES DE LIMA,172179,2025-06-26,FOSTIMON 150UI 1FA PO LIOF 1 AMP DIL 1ML,1.0,169.0,Estimulação,10150,510,18465,RPS,18359,10,EDUARDO LEME ALVES MOTTA,801466,,000000000004719,0.0,0.0,0.0,140.0,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,384495,20250916_153309,Mescladas2.xlsx
4,74126,JULIANA ALVES,74126,JULIANA ALVES DE LIMA,172179,2025-07-24,INDUX 50MG (C.E.),2.0,46.0,Estimulação,10150,502,23020,RPS,22919,10,EDUARDO LEME ALVES MOTTA,801466,,000000000007885,0.0,0.0,0.0,110.0,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,389742,20250916_153309,Mescladas2.xlsx
5,74126,JULIANA ALVES,74126,JULIANA ALVES DE LIMA,172179,2025-07-24,"OESTROGEL PUMP GEL 0,6MG/GC/80G (C.E.)",1.0,40.0,Estimulação,10150,502,23020,RPS,22919,10,EDUARDO LEME ALVES MOTTA,801466,,000000000000531,0.0,0.0,0.0,44.0,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,389743,20250916_153309,Mescladas2.xlsx
6,74126,JULIANA ALVES,74126,JULIANA ALVES DE LIMA,172179,2025-08-01,MENSALIDADE CRIOPRESERVACAO DE OOCITOS -,1.0,65.5,Criopreservação,10150,502,24226,RPS,24128,0,CRIOPRESERVACAO,801466,,S3566.40,0.0,0.0,0.0,65.5,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,391186,20250916_153309,Mescladas2.xlsx
